In [1]:
import pandas as pd

df = pd.read_csv("/home/venkey/Downloads/fake-news/train.csv")

In [3]:
df.isnull().sum()


id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df = df.dropna()


In [8]:
x = df.drop(['label'], axis=1)

In [9]:
y= df['label']


In [11]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [12]:
import tensorflow as tf


2024-07-25 12:07:30.352642: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [14]:
## vocabulary size
voc_size=500

In [15]:
msg = x.copy()

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /home/venkey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
msg.reset_index(inplace=True)

In [19]:
## data preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(msg)):
    review = re.sub('[^a-zA-Z]', ' ', msg['title'][i])
    review= review.lower()
    review =review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words("english")]
    review = ' '.join(review)
    corpus.append(review)


In [21]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [22]:
# embedding repesentation
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding= 'pre', maxlen=sent_length)

In [24]:
#creating model
embedded_feature_vector = 40
Model = Sequential()
Model.add(Embedding(voc_size, embedded_feature_vector, input_length=sent_length))
Model.add(Bidirectional(LSTM(100)))
Model.add(Dense(1, activation='sigmoid'))
Model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
import numpy as np
x_final = np.array(embedded_docs)
y_final =np.array(y)

In [26]:
## traintest split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test  = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [27]:
#model training
Model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)


Epoch 1/10
383/383 [==============================] - 9s 18ms/step - loss: 0.2887 - accuracy: 0.8731 - val_loss: 0.2464 - val_accuracy: 0.8986
Epoch 2/10
383/383 [==============================] - 5s 12ms/step - loss: 0.2200 - accuracy: 0.9073 - val_loss: 0.2315 - val_accuracy: 0.9001
Epoch 3/10
383/383 [==============================] - 6s 15ms/step - loss: 0.2011 - accuracy: 0.9138 - val_loss: 0.2530 - val_accuracy: 0.8996
Epoch 4/10
383/383 [==============================] - 6s 17ms/step - loss: 0.1880 - accuracy: 0.9202 - val_loss: 0.2297 - val_accuracy: 0.9042
Epoch 5/10
383/383 [==============================] - 8s 21ms/step - loss: 0.1782 - accuracy: 0.9254 - val_loss: 0.2200 - val_accuracy: 0.9062
Epoch 6/10
383/383 [==============================] - 7s 19ms/step - loss: 0.1683 - accuracy: 0.9300 - val_loss: 0.2437 - val_accuracy: 0.9062
Epoch 7/10
383/383 [==============================] - 6s 15ms/step - loss: 0.1589 - accuracy: 0.9349 - val_loss: 0.2503 - val_accuracy: 0.9027

In [28]:
y_pred = Model.predict(x_test)

189/189 [==============================] - 1s 4ms/step


In [29]:
y_pred = np.where(y_pred > 0.5 ,1, 0)


In [30]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[3027  392]
 [ 226 2390]]


In [31]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))


0.8975973487986744


In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.89      0.91      3419
           1       0.86      0.91      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.89      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

